In [ ]:
import os
os.makedirs('/content/cache/')
os.makedirs('/content/cache/datasets')
os.makedirs('/content/cache/models')

%env HF_HOME=/content/cache
%env HF_DATASETS_CACHE=/content/cache/datasets
%env TRANSFORMERS_CACHE=/content/cache/models
%env TORCH_HOME=/content/cache

env: HF_HOME=/content/cache
env: HF_DATASETS_CACHE=/content/cache/datasets
env: TRANSFORMERS_CACHE=/content/cache/models
env: TORCH_HOME=/content/cache


In [ ]:
!pip install datasets
!pip install evaluate
!pip install huggingface_hub
!pip install jiwer
!pip install transformers[torch]
!pip install accelerate -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 30.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requi

In [ ]:
from datasets import load_dataset, DatasetDict
from transformers import WhisperFeatureExtractor
from transformers import WhisperTokenizer
from transformers import WhisperProcessor
from datasets import Audio
from transformers import WhisperForConditionalGeneration
import evaluate
from transformers import Seq2SeqTrainingArguments
from transformers import Seq2SeqTrainer

import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

from huggingface_hub import login
from datasets import Dataset

login()
access_token = "yyyyyyyyy"

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [ ]:
# Dataset link
dataset_link = "nilc-nlp/CORAA-MUPE-ASR"

# Distilled model link / starting point
#distilled_model_link = "RodrigoLimaRFL/distil-whisper-nurc-sp-fine-tuned"
#distilled_model_link = "RodrigoLimaRFL/distil-large-nurc-sp"
distilled_model_link = "sidleal/distil-whisper-coraa-mupe-asr-1"

# New fine-tuned model name
fine_tuned_model_name = "distil-whisper-coraa-mupe-asr-2"

# kwargs to define the model
kwargs = {
    "dataset_tags": "sidleal/CORAA-MUPE-ASR-1",
    "dataset": "CORAA-MUPE-ASR",  # a 'pretty' name for the training dataset
    "dataset_args": "split: test",
    "language": "pt",
    "model_name": "CORAA-MUPE-ASR distil-whisper fine-tuned",  # a 'pretty' name for your model
    "finetuned_from": "RodrigoLimaRFL/distil-large-nurc-sp",
    "tasks": "automatic-speech-recognition",
}

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch


In [ ]:
from pathlib import Path

#from datasets import IterableDatasetDict

#import datasets
#datasets.config.DOWNLOADED_DATASETS_PATH = Path("/content")

#mupe_asr = IterableDatasetDict()
mupe_asr = DatasetDict()

#mupe_asr["train"] = load_dataset(
#    dataset_link, split="train+validation", streaming=True
#)
#mupe_asr["test"] = load_dataset(
#    dataset_link, split="test", streaming=True
#)
mupe_asr["train"] = load_dataset(
    dataset_link, split="train"
)

mupe_asr["validation"] = load_dataset(
    dataset_link, split="validation"
)

print(mupe_asr)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Loading dataset shards:   0%|          | 0/66 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['audio_id', 'audio_name', 'file_path', 'speaker_type', 'speaker_code', 'speaker_gender', 'audio_quality', 'start_time', 'end_time', 'duration', 'normalized_text', 'original_text', 'audio'],
        num_rows: 276881
    })
    validation: Dataset({
        features: ['audio_id', 'audio_name', 'file_path', 'speaker_type', 'speaker_code', 'speaker_gender', 'audio_quality', 'start_time', 'end_time', 'duration', 'normalized_text', 'original_text', 'audio'],
        num_rows: 9894
    })
})


In [ ]:
mupe_asr2 = mupe_asr.select_columns(["audio", "original_text"])

feature_extractor = WhisperFeatureExtractor.from_pretrained(distilled_model_link)

tokenizer = WhisperTokenizer.from_pretrained(distilled_model_link, language="Portuguese", task="transcribe")

input_str = mupe_asr2["train"][0]["original_text"]
#input_str = next(iter(mupe_asr2["train"]))["original_text"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")

processor = WhisperProcessor.from_pretrained(distilled_model_link, language="Portuguese", task="transcribe")

mupe_asr3 = mupe_asr2.cast_column("audio", Audio(sampling_rate=16000))

print(mupe_asr3["train"][0])

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Input:                 Elas tinham já uma certa idade, então elas criavam, criaram novos núcleos de amizade e ela,
Decoded w/ special:    <|startoftranscript|><|pt|><|transcribe|><|notimestamps|>Elas tinham já uma certa idade, então elas criavam, criaram novos núcleos de amizade e ela,<|endoftext|>
Decoded w/out special: Elas tinham já uma certa idade, então elas criavam, criaram novos núcleos de amizade e ela,
Are equal:             True


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


{'audio': {'path': 'train/pc_ma_hv273/pc_ma_hv273_346_2280.4359999999997_2289.3199999999997.wav', 'array': array([-0.08056641, -0.04632568, -0.01147461, ..., -0.07089233,
       -0.07369995, -0.07662964]), 'sampling_rate': 16000}, 'original_text': 'Elas tinham já uma certa idade, então elas criavam, criaram novos núcleos de amizade e ela,'}


In [ ]:
!gsutil cp gs://usp_pos_doc_sid/mupe/models/2-checkpoint-5000.tar.gz .
!tar -xzf 2-checkpoint-5000.tar.gz

In [ ]:
def prepare_dataset(batch):
    # load audio data
    audio = batch["audio"]

    # compute input length
    batch["input_length"] = len(batch["audio"])

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["original_text"]).input_ids

    # compute labels length
    batch["labels_length"] = len(tokenizer(batch["original_text"], add_special_tokens=False).input_ids)

    return batch

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}



#model = WhisperForConditionalGeneration.from_pretrained(distilled_model_link)
model = WhisperForConditionalGeneration.from_pretrained("/content/checks/checkpoint-5000")

model.generation_config.language = "portuguese"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None




MAX_DURATION_IN_SECONDS = 30.0
max_input_length = MAX_DURATION_IN_SECONDS * 16000

def filter_inputs(input_length):
    """Filter inputs with zero input length or longer than 30s"""
    return 0 < input_length < max_input_length

max_label_length = model.config.max_length

def filter_labels(labels_length):
    """Filter label sequences longer than max length (448)"""
    return labels_length < max_label_length

mupe_asr4 = mupe_asr3.map(prepare_dataset, remove_columns= mupe_asr3.column_names["train"])

mupe_asr4 = mupe_asr4.filter(filter_inputs, input_columns=["input_length"])

mupe_asr4 = mupe_asr4.filter(filter_labels, input_columns=["labels_length"])

#mupe_asr4 = mupe_asr3.map(prepare_dataset, remove_columns=mupe_asr3.column_names["train"], num_proc=1)


data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

metric = evaluate.load("wer")


In [ ]:
mupe_asr4 = mupe_asr4.filter(filter_inputs, input_columns=["input_length"])

mupe_asr4 = mupe_asr4.filter(filter_labels, input_columns=["labels_length"])

#mupe_asr4 = mupe_asr3.map(prepare_dataset, remove_columns=mupe_asr3.column_names["train"], num_proc=1)


data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

metric = evaluate.load("wer")


Filter:   0%|          | 0/276881 [00:00<?, ? examples/s]

Filter:   0%|          | 0/9894 [00:00<?, ? examples/s]

Filter:   0%|          | 0/276881 [00:00<?, ? examples/s]

Filter:   0%|          | 0/9894 [00:00<?, ? examples/s]

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./" + fine_tuned_model_name,  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=18000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=3000,
    eval_steps=3000,
    logging_steps=500,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
    #num_train_epochs=5,
)

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=mupe_asr4["train"],
    eval_dataset=mupe_asr4["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

trainer.train()

trainer.push_to_hub(**kwargs)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Wer
3000,0.332200,0.368694,19.851412
6000,0.324500,0.346568,18.995061
9000,0.302100,0.332036,18.040911
12000,0.285200,0.322046,18.069675
15000,0.281900,0.309504,17.408098
18000,0.206200,0.306652,16.906779


The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'begin_suppress_tokens': [220, 50257]}
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more 

CommitInfo(commit_url='https://huggingface.co/sidleal/distil-whisper-coraa-mupe-asr-2/commit/6b0d8b8a525e792aaa5bda23092e0ca16a579151', commit_message='End of training', commit_description='', oid='6b0d8b8a525e792aaa5bda23092e0ca16a579151', pr_url=None, pr_revision=None, pr_num=None)